In [2]:
from itertools import product
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import (MaxAbsScaler, MinMaxScaler,
                                   PolynomialFeatures, RobustScaler,
                                   StandardScaler)
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import (BaseCrossValidator, GridSearchCV, KFold,
                                     RandomizedSearchCV, StratifiedKFold,
                                     check_cv, train_test_split)
from sklearn.linear_model import (ARDRegression, BayesianRidge, ElasticNet,
                                  ElasticNetCV, Lars, Lasso, LassoLars,
                                  LinearRegression, LogisticRegression,
                                  LogisticRegressionCV,
                                  OrthogonalMatchingPursuit, Ridge)
from sklearn.ensemble import (GradientBoostingClassifier,
                              GradientBoostingRegressor,
                              RandomForestClassifier, RandomForestRegressor)
from sklearn.base import BaseEstimator, is_regressor
from econml.dr import DRLearner
from econml.dml import CausalForestDML, KernelDML, LinearDML, SparseLinearDML
import sklearn.preprocessing
import sklearn.neural_network
import sklearn.linear_model
import sklearn.ensemble
import sklearn
import argparse
import logging
import os
import pdb
import numpy as np
import pandas as pd
import pickle
import utils
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import time
import warnings
import csv
import os
import during_crossfit_params

During Cross-fitting

In [ ]:
data, X, T, Y, true_ITE, true_ATE, true_ATE_stderr, is_discrete = utils.load_ihdp()
# X, X_val, T, T_val, Y, Y_val = train_test_split(X, T, Y, train_size=0.6,shuffle=True, random_state=123)
# X_val, X_test, T_val, T_test, Y_val, Y_test = train_test_split(X_val, T_val, Y_val, train_size=.5, shuffle=True, random_state=123)

In [ ]:
ci_estimator_list = ['tl', 'dml', 'kernel_dml', 'CausalForestDML']
model_y = ['linear', 'forest', 'gbf', 'nnet']
model_t = ['linear', 'forest', 'gbf', 'nnet']

In [ ]:
k = 4
fold_models, total_run_time = during_crossfit_params.get_models_during_k_folds(X, T, Y, k, ci_estimator_list, model_y, model_t, model_y_discrete=False, model_t_discrete=True)
print(fold_models, total_run_time)

{'fold 0': {'model_y': ElasticNetCV(l1_ratio=0.9, max_iter=100), 'model_t': LogisticRegressionCV(Cs=1, max_iter=25), 'Mse': {'tl': 7.522415610112231, 'dml': 6.482037995026961, 'kernel_dml': 5.205798775508355, 'CausalForestDML': 6.003187772347634}, 'Runtime': {'tl': 0.1557145118713379, 'dml': 9.280638694763184, 'kernel_dml': 0.260221004486084, 'CausalForestDML': 0.48456573486328125}}, 

'fold 1': {'model_y': RandomForestRegressor(max_depth=10, min_samples_split=10, n_estimators=25), 'model_t': RandomForestClassifier(max_depth=5, min_samples_leaf=2, n_estimators=25), 'Mse': {'tl': 8.377986344419789, 'dml': 7.620422451900242, 'kernel_dml': 5.1935256107797425, 'CausalForestDML': 5.747076065870897}, 'Runtime': {'tl': 0.13500022888183594, 'dml': 1.3679900169372559, 'kernel_dml': 0.39501476287841797, 'CausalForestDML': 0.4939577579498291}}, 

'fold 2': {'model_y': GradientBoostingRegressor(), 'model_t': GradientBoostingClassifier(learning_rate=0.01), 'Mse': {'tl': 8.65248748763551, 'dml': 6.426960624988473, 'kernel_dml': 4.387014823628399, 'CausalForestDML': 5.354179619841489}, 'Runtime': {'tl': 0.14799737930297852, 'dml': 2.4449832439422607, 'kernel_dml': 0.47730207443237305, 'CausalForestDML': 0.5939912796020508}}, 

'fold 3': {'model_y': MLPRegressor(alpha=0.001, max_iter=25), 'model_t': MLPClassifier(alpha=0.001, learning_rate='adaptive', max_iter=25), 'Mse': {'tl': 14.650894172023754, 'dml': 8.159747525064809, 'kernel_dml': 4.545129616395722, 'CausalForestDML': 6.8417017911365585}, 'Runtime': {'tl': 0.25336551666259766, 'dml': 0.7475666999816895, 'kernel_dml': 0.4789872169494629, 'CausalForestDML': 0.5972983837127686}}} 

47.82886266708374


In [ ]:
mse_all_estimators, best_mse_fold, best_models_mse, runtime_all_estimators, best_runtime_fold, best_models_time = during_crossfit_params.find_best_fold_mse_runtime(fold_models, ci_estimator_list)

In [ ]:
mse_all_estimators, best_mse_fold, best_models_mse

({'tl': [7.522415610112231,
   8.377986344419789,
   8.65248748763551,
   14.650894172023754],
  'dml': [6.482037995026961,
   7.620422451900242,
   6.426960624988473,
   8.159747525064809],
  'kernel_dml': [5.205798775508355,
   5.1935256107797425,
   4.387014823628399,
   4.545129616395722],
  'CausalForestDML': [6.003187772347634,
   5.747076065870897,
   5.354179619841489,
   6.8417017911365585]},
 {'tl': 0, 'dml': 2, 'kernel_dml': 2, 'CausalForestDML': 2},
 {'tl': {'best_model_y': ElasticNetCV(l1_ratio=0.9, max_iter=100),
   'best_model_t': '',
   'Mse': 7.522415610112231},
  'dml': {'best_model_y': GradientBoostingRegressor(),
   'best_model_t': GradientBoostingClassifier(learning_rate=0.01),
   'Mse': 6.426960624988473},
  'kernel_dml': {'best_model_y': GradientBoostingRegressor(),
   'best_model_t': GradientBoostingClassifier(learning_rate=0.01),
   'Mse': 4.387014823628399},
  'CausalForestDML': {'best_model_y': GradientBoostingRegressor(),
   'best_model_t': GradientBoostingC

In [ ]:
runtime_all_estimators, best_runtime_fold, best_models_time

({'tl': [0.1557145118713379,
   0.13500022888183594,
   0.14799737930297852,
   0.25336551666259766],
  'dml': [9.280638694763184,
   1.3679900169372559,
   2.4449832439422607,
   0.7475666999816895],
  'kernel_dml': [0.260221004486084,
   0.39501476287841797,
   0.47730207443237305,
   0.4789872169494629],
  'CausalForestDML': [0.48456573486328125,
   0.4939577579498291,
   0.5939912796020508,
   0.5972983837127686]},
 {'tl': 1, 'dml': 3, 'kernel_dml': 0, 'CausalForestDML': 0},
 {'tl': {'best_model_y': RandomForestRegressor(max_depth=10, min_samples_split=10, n_estimators=25),
   'best_model_t': '',
   'Runtime': 0.13500022888183594},
  'dml': {'best_model_y': MLPRegressor(alpha=0.001, max_iter=25),
   'best_model_t': MLPClassifier(alpha=0.001, learning_rate='adaptive', max_iter=25),
   'Runtime': 0.7475666999816895},
  'kernel_dml': {'best_model_y': ElasticNetCV(l1_ratio=0.9, max_iter=100),
   'best_model_t': LogisticRegressionCV(Cs=1, max_iter=25),
   'Runtime': 0.260221004486084},


In [ ]:
key = 'ihdp'
results_file = f'../results/{key}_during_crossfit_params.csv'

# if not os.path.exists(results_file):
#     os.makedirs(results_file)

In [ ]:
# Write MSE and runtime data to CSV file
with open(results_file, mode='w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Estimator', 'Fold', 'model_y', 'model_t', 'MSE', 'Runtime', 'Best first-stage models Mse', 'Best first-stage models Runtime', 'Best Mse', 'Best Runtime'])

    for estimator in mse_all_estimators:
        for i in range(k):
            mse = mse_all_estimators[estimator][i]
            runtime = runtime_all_estimators[estimator][i]
            csvwriter.writerow([estimator, i, fold_models[f'fold {i}']['model_y'], fold_models[f'fold {i}']['model_t'], mse, runtime, '', '', '', ''])
            i += 1

        best_mse = best_models_mse[estimator]['Mse']
        best_runtime = best_models_time[estimator]['Runtime']

        best_mse_models = f"{best_models_mse[estimator]['best_model_y']}\n\n{best_models_mse[estimator]['best_model_t']}"

        best_runtime_models = f"{best_models_time[estimator]['best_model_y']}\n\n{best_models_time[estimator]['best_model_t']}"

        csvwriter.writerow([estimator, f'Best Mse, Runtime Fold - {best_mse_fold[estimator]}, {best_runtime_fold[estimator]}', '', '', '', '', best_mse_models, best_runtime_models, best_mse, best_runtime])

After refactoring

In [3]:
ci_estimator_list = ['tl', 'dml', 'kernel_dml', 'CausalForestDML']
model_y = ['linear', 'forest', 'gbf', 'nnet']
model_t = ['linear', 'forest', 'gbf', 'nnet']

key = 'ihdp'
during_crossfit_params.main(ci_estimator_list, model_y, model_t, key, k=4)